In [1]:
import torch
from huggingface_hub import Repository

In [2]:
path_to_home = "../"
#path_to_home = "./drive/MyDrive/receiptlayoutlm/"

In [3]:
hf_repo = Repository(path_to_home + "hugging_face_repo", clone_from="https://huggingface.co/sibrun/receiptlayoutlm")

/Users/simon/PycharmProjects/receiptlayoutlm/code/../hugging_face_repo is already a clone of https://huggingface.co/sibrun/receiptlayoutlm. Make sure you pull the latest changes with `repo.git_pull()`.


In [4]:
hf_repo = Repository(path_to_home + "hugging_face_repo")
#hf_repo.git_pull()

In [5]:
from datasets import load_dataset

ds_receipts = load_dataset("sibrun/receipts", use_auth_token=True)

{'image': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='uint8', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'bbox': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [4]:
from datasets import load_from_disk

ds_receipts = load_from_disk(path_to_home + "datasets/ds_receipts_final")

In [5]:
from datasets import Features, Array2D, Array3D, Sequence, Value
max_length = 512
features = Features({
    'image': Array3D(dtype="uint8", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype="int32"), length=max_length),
    'bbox': Array2D(dtype="int64", shape=(max_length, 4)),
    'labels': Sequence(feature=Value(dtype="int64"), length=max_length),
    'attention_mask': Sequence(feature=Value(dtype="int8"), length=max_length),
})

In [6]:
ds_receipts = ds_receipts.cast(features)
ds_receipts['train'].features

Loading cached processed dataset at ../datasets/ds_receipts_final/train/cache-e5c271fb43f54ac8.arrow
Loading cached processed dataset at ../datasets/ds_receipts_final/test/cache-c1bc42bb40029ecf.arrow


{'image': Array3D(shape=(3, 224, 224), dtype='uint8', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=512, id=None),
 'bbox': Array2D(shape=(512, 4), dtype='int64', id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=512, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=512, id=None)}

In [7]:
label_names = ['company', 'date', 'address', 'total']
labels = ['O'] + label_names
num_labels = len(labels)
ids_to_labels = {k: v for k, v in enumerate(labels)}
labels_to_ids = {v: k for k, v in enumerate(labels)}

In [8]:
from transformers import AutoConfig
from transformers import LayoutLMv2ForTokenClassification

xlm_config = AutoConfig.from_pretrained("microsoft/layoutxlm-base",
                                         num_labels=num_labels,
                                         id2label=ids_to_labels,
                                         label2id=labels_to_ids)

In [10]:
model_xlm = LayoutLMv2ForTokenClassification.from_pretrained("microsoft/layoutxlm-base", config=xlm_config)

Some weights of the model checkpoint at microsoft/layoutxlm-base were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.0.shortcut.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.15.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.10.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.0.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.22.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.8.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.19.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.1.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.3.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.18.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.1.conv1.norm.num_batches_tracked', 'l

In [11]:
model_xlm.save_pretrained(path_to_home + "models/token_class_xlm_base")

In [12]:
model_xlm = LayoutLMv2ForTokenClassification.from_pretrained(path_to_home + "models/token_class_xlm_base")

In [9]:
model_xlm = LayoutLMv2ForTokenClassification.from_pretrained(path_to_home + "models/receiptlayoutlm")

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ds_receipts.set_format(type="torch", device=device)
model_xlm.to(device)

LayoutLMv2ForTokenClassification(
  (layoutlmv2): LayoutLMv2Model(
    (embeddings): LayoutLMv2Embeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (x_position_embeddings): Embedding(1024, 128)
      (y_position_embeddings): Embedding(1024, 128)
      (h_position_embeddings): Embedding(1024, 128)
      (w_position_embeddings): Embedding(1024, 128)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (visual): LayoutLMv2VisualBackbone(
      (backbone): FPN(
        (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
        (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 

In [11]:
from argparse import Namespace

config = {"train_batch_size": 16,
          "valid_batch_size": 8,
          "weight_decay": 0.1,
          "learning_rate": 5e-5,
          "num_train_epochs": 4,
          "seed": 1,
          "eval_steps": 25,
          "save_checkpoint_steps": 100}

args = Namespace(**config)

logging_file_path = path_to_home + "log/logging/train.log"
wandb_dir = path_to_home +"log/wandb"
tensorboard_dir = path_to_home + "log/tensorboard"
project_name = "receiptlayoutlm"
user_name = "sibrun"
hf_model_path = path_to_home + "models/" + project_name
hf_model_name = user_name + "/" + project_name

In [12]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(ds_receipts['train'], batch_size=args.train_batch_size, shuffle=True)
eval_dataloader = DataLoader(ds_receipts['test'], batch_size=args.valid_batch_size)

In [13]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k, v.shape)

image torch.Size([16, 3, 224, 224])
input_ids torch.Size([16, 512])
bbox torch.Size([16, 512, 4])
labels torch.Size([16, 512])
attention_mask torch.Size([16, 512])


In [14]:
optimizer = torch.optim.AdamW(model_xlm.parameters(), lr=args.learning_rate)

In [19]:
from tensorboardX import SummaryWriter
import logging
#import wandb
#import transformers
#import datasets

def setup_logging():
    logger = logging.getLogger(__name__)
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
        handlers=[logging.FileHandler(logging_file_path), logging.StreamHandler()])
    #wandb.init(project=project_name_wandb,
    #           config=args,
    #           dir=wandb_dir,
    #           settings=wandb.Settings(start_method="fork"),
    #           entity="sibrun",
    #           sync_tensorboard=True)
    #run_name = wandb.run.name
    tb_writer = SummaryWriter(log_dir=tensorboard_dir)
    tb_writer.add_hparams(vars(args), {'0': 0})
    logger.setLevel(logging.INFO)
    #datasets.utils.logging.set_verbosity_debug()
    #transformers.utils.logging.set_verbosity_info()
    return logger, tb_writer #, run_name

def log_scores(step, scores, tb_writer):
    logger.info(f"Step {step}: {scores}")
    #wandb.log(scores)
    [tb_writer.add_scalar(k, v, step) for k, v in scores.items()]

In [41]:
from datasets import load_metric
import numpy as np
import collections, functools, operator

seqeval_metrics = load_metric("seqeval")

def compute_metrics(predictions, labels, metrics=seqeval_metrics):
    scores = metrics.compute(predictions=predictions, references=labels)
    overall_scores = {'precision': scores['overall_precision'],
                      'recall':  scores['overall_recall'],
                      'f1': scores['overall_f1'],
                      'accuracy': scores['overall_accuracy']}
    return overall_scores

def evaluate():
    model_xlm.eval()
    loss_list = []
    scores_list = []
    for step, batch in enumerate(eval_dataloader):
        if step > 1:
            break
        with torch.no_grad():
            model_outputs = model_xlm(**batch)
        loss_list.append(model_outputs.loss)
        print(model_outputs.logits.shape)
        predictions = np.argmax(model_outputs.logits, axis=2)
        print(predictions)
        labels = batch['labels']
        print(labels)
        mask = batch['attention_mask']
        print(mask)
        scores = compute_metrics(predictions, labels)
        print(scores)
        scores_list.append(scores)
    total_loss = torch.mean(torch.stack(loss_list))
    total_scores = dict(functools.reduce(operator.add, map(collections.Counter, scores_list)))
    try:
        perplexity = torch.exp(total_loss)
    except OverflowError:
            perplexity = torch.tensor(float("inf"))
    total_scores['loss/eval'] = total_loss.item()
    total_scores['perplexity'] = perplexity.item()
    return total_scores

In [23]:
from random import seed
seed(args.seed)

logger, tb_writer = setup_logging()

In [34]:
step = 0
model_xlm.train()
for epoch in range(args.num_train_epochs):
   log_scores(step, {'epoch': epoch}, tb_writer)
   for batch in train_dataloader:
       optimizer.zero_grad()
       outputs = model_xlm(**batch)
       loss = outputs.loss
       log_scores(step, {'loss/train': loss.item()}, tb_writer)
       if step % args.eval_steps == 0:
           logger.info('Evaluating')
           scores = evaluate()
           log_scores(step, scores, tb_writer)
       if step % args.save_checkpoint_steps == 0 and step > 0:
           logger.info('Saving model checkpoint')
           model_xlm.save_pretrained(hf_model_path)
           #model_xlm.push_to_hub(hf_model_name, commit_message=f'step {step}')
       model_xlm.train()
       loss.backward()
       optimizer.step()
       step += 1

logger.info('Evaluating and saving model after training')
scores = evaluate()
log_scores(step, scores, tb_writer)
model_xlm.save_pretrained(hf_model_path)
model_xlm.push_to_hub(hf_model_name, commit_message='final model')

03/14/2022 15:32:43 - INFO - __main__ - Step 0: {'epoch': 0}


KeyboardInterrupt: 

In [42]:
scores = evaluate()

torch.Size([8, 512, 5])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[-100,    0, -100,  ..., -100, -100, -100],
        [-100,    0, -100,  ..., -100, -100, -100],
        [-100,    0, -100,  ..., -100, -100, -100],
        ...,
        [-100,    0, -100,  ..., -100, -100, -100],
        [-100,    0, -100,  ..., -100, -100, -100],
        [-100,    0,    0,  ..., -100, -100, -100]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': 0.21875}
torch.Size([8, 512, 5])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [